# Домашнее задание 2. По чуть-чуть про pandas, sklearn, kNN

В этом задании вы в миниатюре пройдете цикл решения задачи анализа данных от предобработки данных до построения модели и оценки качества. Вы будете решать задачу кредитного скоринга (вернет клиент кредит или нет) на примере небольшой выборки данных. 

В процессе решения задания вы поближе познакомитесь с библиотеками pandas, sklearn и тем, как работают классы в python, а также немного вспомните numpy, самостоятельно реализовав 1NN (это очень просто, потому что сам алгоритм очень простой).

Все задачи нужно выполнять исключительно самостоятельно! Если что-то непонятно, задавайте вопросы в чате. При обнаружении списывания ноль за задание ставится и тому, кто списал, и тому, у кого списали.

Также во всем задании запрещено использование циклов (исключение - второе бонусное задание)! Все операции необходимо выполнять средствами numpy, pandas и sklearn.

Разделение выборки на обучение и контроль производится случайно (если только вы не указали конкретный random_state), поэтому результаты при разных запусках ноутбука могут получаться разные.

Считывание данных уже сделано:

In [1]:
import numpy as np
import pandas as pd

In [2]:
tab = pd.read_csv("https://raw.githubusercontent.com/nadiinchi/iad2019/master/materials/scoring.csv")

Выведите первые несколько строчек датасета, посмотрите названия колонок:

In [7]:
tab.head()

,account,duration,credit_history,purpose,amount,savings,employment,installment_rate,guarantors,residence,...,job,maintenance_people,telephone,foreign,real_estate,life_insurance,car,housing_rent,sex,target
0,0,18,4,furniture,3780,1,1,3,1,2,...,3,1,1,yes,0,0,1,0,male,1
1,1,24,2,car_new,1246,1,1,4,1,2,...,1,1,0,yes,1,0,0,0,male,0
2,0,15,2,car_used,3029,1,3,2,1,2,...,2,1,0,yes,0,0,1,0,male,1
3,0,36,3,television,4463,1,2,4,1,2,...,3,1,1,yes,0,0,1,0,male,0
4,-1,27,2,television,3416,1,2,3,1,2,...,3,1,0,yes,0,0,1,0,male,1


Сначала несколько вопросов просто так:
    
 __[0.5 балла]__ Какой процент клиентов моложе 30 лет (колонка age) снимает жилье (колонка housing_rent)? А какой процент клиентов старше 40 лет снимает жилье?

In [9]:
len(tab[(tab.age<30) & (tab.housing_rent==1)])/len(tab.housing_rent)

0.12166666666666667

 __[0.5 балла]__ У скольких иностранцев (foreign == 'yes') число кредитов (credits_number) больше 2?

In [10]:
len(tab[(tab.foreign=='yes') & (tab.credits_number>=2)])

188

 __[0.5 балла]__ Выведите колонки job, life_insurance и guarantors для клиентов с номерами 105, 158, 306 и 336 одной таблицей (воспользуйтесь индексацией):

In [23]:
tab.loc[[105,158,306,336],['job','life_insurance','guarantors']]

,job,life_insurance,guarantors
105,2,1,1
158,2,0,1
306,3,0,0
336,2,1,1


 __[0.5 балла]__ Выведите среднюю сумму кредита (колонка amount) в каждой категории (колонка purpose), воспользовавшись методом groupby:

In [38]:
tab.groupby(('purpose')).amount.mean()

purpose
business               4405.098361
car_new                3156.796178
car_used               5690.403509
domestic_appliances    1307.875000
education              3276.258065
furniture              3312.677966
others                 8681.700000
repairs                2792.769231
retraining             1156.000000
television             2554.237762
Name: amount, dtype: float64

Теперь по существу: нужно подготовить данные к построению модели.

 __[0.5 балла]__ Как вы видите, не все данные являются числовыми. С признаком purpose мы работать не будем - удалите его из датафрейма:

In [70]:
df=tab.drop('purpose',axis=1)

 __[0.5 балла]__ Необходимо перекодировать оставшиеся строковые значения в числа (закодировать пол female->0 и male->1 и колонку foreign числами yes->1 и no->0). Сделайте это с помощью pandas (без циклов, конечно же):

In [71]:
df.sex=df.sex.map(dict(male=1,female=0))
df.foreign=df.foreign.map(dict(yes=1,no=0))
df.head()

,account,duration,credit_history,amount,savings,employment,installment_rate,guarantors,residence,age,...,job,maintenance_people,telephone,foreign,real_estate,life_insurance,car,housing_rent,sex,target
0,0,18,4,3780,1,1,3,1,2,35,...,3,1,1,1,0,0,1,0,1,1
1,1,24,2,1246,1,1,4,1,2,23,...,1,1,0,1,1,0,0,0,1,0
2,0,15,2,3029,1,3,2,1,2,33,...,2,1,0,1,0,0,1,0,1,1
3,0,36,3,4463,1,2,4,1,2,26,...,3,1,1,1,0,0,1,0,1,0
4,-1,27,2,3416,1,2,3,1,2,27,...,3,1,0,1,0,0,1,0,1,1


 __[0.5 балла]__ Целевая переменная хранится в колонке target (последняя колонка). Являются ли классы сбалансированными (одинаковое число 0 и 1)? Воспользуйтесь методом [value_counts](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.value_counts.html).

In [72]:
df.target.value_counts()

1    300
0    300
Name: target, dtype: int64

Выведем статистику датасета:

In [73]:
df.describe()

,account,duration,credit_history,amount,savings,employment,installment_rate,guarantors,residence,age,...,job,maintenance_people,telephone,foreign,real_estate,life_insurance,car,housing_rent,sex,target
count,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,...,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000
mean,0.083333,21.878333,2.410000,3470.491667,1.200000,2.348333,2.988333,0.901667,2.831667,35.181667,...,1.906667,1.150000,0.391667,0.965000,0.268333,0.205000,0.343333,0.186667,0.598333,0.500000
std,0.915306,12.749161,1.098747,3090.052187,0.924531,1.208069,1.131517,0.298013,1.103945,11.416182,...,0.662334,0.357369,0.488530,0.183933,0.443462,0.404038,0.475218,0.389969,0.490644,0.500417
min,-1.000000,4.000000,0.000000,250.000000,0.000000,0.000000,1.000000,0.000000,1.000000,19.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-1.000000,12.000000,2.000000,1363.250000,1.000000,2.000000,2.000000,1.000000,2.000000,26.000000,...,2.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,18.000000,2.000000,2347.000000,1.000000,2.000000,3.000000,1.000000,3.000000,33.000000,...,2.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.500000
75%,1.000000,27.000000,4.000000,4318.750000,1.000000,3.000000,4.000000,1.000000,4.000000,41.000000,...,2.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000
max,2.000000,72.000000,4.000000,18424.000000,4.000000,4.000000,4.000000,1.000000,4.000000,75.000000,...,3.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


 __[0.5 балла]__ Проанализируйте таблицу. Есть ли признаки с пропущенными значениями? (ответ необходимо писать в этой же ячейке; к ответу сделайте пояснение)

In [75]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 21 columns):
account               600 non-null int64
duration              600 non-null int64
credit_history        600 non-null int64
amount                600 non-null int64
savings               600 non-null int64
employment            600 non-null int64
installment_rate      600 non-null int64
guarantors            600 non-null int64
residence             600 non-null int64
age                   600 non-null int64
credits_number        600 non-null int64
job                   600 non-null int64
maintenance_people    600 non-null int64
telephone             600 non-null int64
foreign               600 non-null int64
real_estate           600 non-null int64
life_insurance        600 non-null int64
car                   600 non-null int64
housing_rent          600 non-null int64
sex                   600 non-null int64
target                600 non-null int64
dtypes: int64(21)
memory usage: 98

 __[0.5 балла]__ Запишите в переменную X матрицу объекты-признаки (выделите все колонки признаков, кроме target), а в переменную y - вектор правильных ответов (колонка target). К обоим объектам припишите .values, чтобы "достать" numpy-матрицу из датафрейма: далее мы везде будем работать с numpy-матрицами.

In [76]:
X=df.drop('target',axis=1)
y=df['target']

 __[0.5 балла]__ С помощью sklearn-функции train_test_split разделите данные на обучение и контроль (пропорцию оставьте по умолчанию). Ячейка ?train_test_split покажет документацию функции (с примерами использования), также можно загуглить документацию.

In [77]:
from sklearn.model_selection import train_test_split

In [78]:
?train_test_split

In [122]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.35, random_state=69)

Выведите форму (shape) обучающей и контрольной выборок:

In [123]:
print(X_train.shape,
X_test.shape)

(390, 20) (210, 20)


 __[0.5 балла]__ Как вы видели выше, все колонки имеют разный масштаб, это будет мешать нам при запуске метрического классификатора. Отнормируйте данные с помощью StandardScaler (интерфейс такой же, как мы изучали на семинаре):

In [117]:
from sklearn.preprocessing import StandardScaler

In [118]:
scaler=StandardScaler()

In [131]:
scaler.fit(X_train)
X_train_n=scaler.transform(X_train)


In [133]:

scaler.fit(X_test)
X_test_n=scaler.transform(X_test)

 __[0.5 балла]__ Теперь обучите классификатор 1-NN из sklearn на обучающей выборке, сделайте предсказания отдельно для обучающей и для контрольной выборки, на каждой выборке посчитайте точность (accuracy) с помощью accuracy_score. При создании объекта KNeighborsClassifier необходимо указать число соседей 1 (по умолчанию оно больше). 

Интерфейс KNeighborsClassifier такой же, как мы изучали на семинаре, а accuracy-score принимает на вход вектор предсказаний и вектор правильных ответов.

In [120]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [134]:
knn=KNeighborsClassifier()
knn.fit(X_train_n,y_train)
temp=knn.predict(X_test_n)
accuracy_score(y_test,temp)

0.5952380952380952

In [101]:
accuracy_score?

 __[4 балла,  включая my_accuracy и тестирование реализации]__ Наконец, реализуйте свой 1NN, воспользовавшись шаблоном ниже. Ваш класс должен уметь работать в двух режимах: в евклидовой метрикой и с манхэттанской метрикой (формулы метрик ниже). Напоминаем принцип работы 1NN: чтобы сделать предсказание для объекта, он выбирает ближайший объект из обучения и возвращает такой же класс, как у него.
* в методе \__init\__ необходимо запомнить, с какой метрикой мы работаем
* в методе fit нужно запомнить выборку - больше при обучении kNN ничего делать не нужно
* в методе predict нужно сделать три шага:
    * сначала построить матрицу расстояний между контрольными и обучающими объектами. Это умеет автоматически делать функция cdist, но за ее использование штраф 1.5 балла - лучше посчитать матрицу расстояний самостоятельно с помощью "чистого" numpy
    * затем взять argmin по строкам - получится вектор индексов ближайших обучающих объектов для каждого контрольного объекта
    * выбрать компоненты y_train, отвечающие компонентам вектора индексов. Иными словами, вернуть те же метки классов, что у ближайших обучающих объектов.
    
Формулы метрик (расстояний):
* евклидова: $$\rho(x, y) = \sqrt{\sum_{i=1}^d (x_i - y_i)^2}$$
* манхэттанская: $$\rho(x, y) = \sum_{i=1}^d |x_i - y_i|$$

Модуль в numpy вычисляется как np.abs.

In [ ]:
from scipy.spatial.distance import cdist # за использование этой функции штраф 1.5 балла :)

In [ ]:
class MyKNN:
    def __init__(self, metric="euclid"):
        
        
    def fit(self, X_tr, y_tr):
        
        
    def predict(self, X_te):
        

Реализуйте также функцию вычисления точности - доли правильных ответов:

In [ ]:
def my_accuracy(y_pred, y_test):
    

Пришло время запустить ваш классификатор и вычислить точность с помощью вашей функции. Обучите классификатор, сделайте предсказания для обучающей и контрольной выборок и посчитайте точность в обоих случаях. Сделайте это два раза - ддя евклидовой и манхэттанской метрик. Для евклидовой метрики цифры должны получиться точно такие же, как для реализации sklearn.

### Бонусные задания

#### Бонусное задание 1: реализация kNN (до 3 баллов)
Добавьте в ваш KNN параметр k (число соседей) и реализуйте k-NN, который возвращает самый популярный класс среди k ближайших соседей объекта. Если число ближайших объектов для разных классов совпадает, возвращайте класс с меньшим номером. За реализацию, работающую для любого числа классов - 3 балла (посмотрите функции np.bincount, np.apply_along_axis (к сожалению, bincount не применяется построчно), np.argpartition), за реализацию, работающую только для двух классов - 2 балла. Не забудьте протестировать свой алгоритм :) В этом задании циклы использовать нельзя!


#### Бонусное задание 2: реализация CentroidClassifier (3 балла)
Реализуйте другой метрический классификатор - центроидный. Он работает чуть-чуть сложнее kNN, но зато гораздо эффективнее: на этапе обучения классификатор запоминает центр каждого класса ("средний" объект класса). На этапе предсказания классификатор возвращает тот класс, чей центр ближе. Не забудьте протестировать свой алгоритм :) Метрика - евклидова, для вычисления центра класса нужно найти среднее каждого признака по объектам этого класса. В этом задании можно использовать 1 цикл - цикл по классам при подсчете центров классов.

In [ ]:
class MyCentroid:
    def fit(self, X_tr, y_tr):
        
    def predict(self, X_te):
